<a href="https://colab.research.google.com/github/01star01ek/01star01ek/blob/main/%EC%A0%84%EC%B2%B4%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf /content/results

In [ ]:
# ▶️ Git 클론 및 환경 설치
%cd /content
!git clone https://github.com/woctezuma/SimSwap.git
%cd /content/SimSwap
!git checkout upgrade-insightface

!pip install insightface==0.7.3 onnxruntime moviepy > /dev/null
!pip install imageio==2.34.0 > /dev/null

# ▶️ 모델 및 리소스 다운로드
!wget https://github.com/woctezuma/SimSwap-colab/releases/download/antelope/antelope.zip
!wget -P ./arcface_model https://github.com/woctezuma/SimSwap-colab/releases/download/1.0/arcface_checkpoint.tar
!wget https://github.com/neuralchen/SimSwap/releases/download/1.0/checkpoints.zip
!wget -P ./parsing_model/checkpoint https://github.com/neuralchen/SimSwap/releases/download/1.0/79999_iter.pth
!wget https://github.com/neuralchen/SimSwap/releases/download/512_beta/512.zip

# ▶️ 압축 해제
!unzip -oq checkpoints.zip -d ./checkpoints
!unzip -oq 512.zip -d ./checkpoints
!unzip -oq antelope.zip -d ./insightface_func/models/


/content
Cloning into 'SimSwap'...
remote: Enumerating objects: 1143, done.
remote: Counting objects: 100% (505/505), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 1143 (delta 417), reused 385 (delta 385), pack-reused 638 (from 1)
Receiving objects: 100% (1143/1143), 211.46 MiB | 15.12 MiB/s, done.
Resolving deltas: 100% (601/601), done.
/content/SimSwap
Already on 'upgrade-insightface'
Your branch is up to date with 'origin/upgrade-insightface'.
--2025-06-16 21:50:14--  https://github.com/woctezuma/SimSwap-colab/releases/download/antelope/antelope.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/378697706/282dd755-3c47-466c-b3e5-be6e4d9a491f?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20250616%2Fus-east-1%2Fs3%2Faws4_requ

In [ ]:
# ▶️ SimSwap 경로 설정
%cd /content/SimSwap

import sys
sys.path.append('/content/SimSwap')

import os
import cv2
import torch
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms
from models.models import create_model
from options.test_options import TestOptions
from insightface_func.face_detect_crop_multi import Face_detect_crop
from util.videoswap import video_swap

# ▶️ PyTorch 2.6+ 보안 대응 - 모델 클래스 허용
from torch.serialization import add_safe_globals
from models.arcface_models import ResNet, IRBlock, SEBlock
from torch.nn import (
    Conv2d, BatchNorm2d, BatchNorm1d, PReLU, MaxPool2d, Sequential,
    ReLU, Linear, AdaptiveAvgPool2d, Dropout, Module, Sigmoid
)

add_safe_globals([
    Conv2d, BatchNorm2d, BatchNorm1d, PReLU, MaxPool2d, Sequential,
    ReLU, Linear, AdaptiveAvgPool2d, Dropout, Module, Sigmoid,
    ResNet, IRBlock, SEBlock
])


/content/SimSwap


  if event.key is 'enter':



In [ ]:
# ▶️ 기본 설정
crop_size = 512  # 정확도 유지용
det_thresh = 0.5
det_size = (320, 320)

# ▶️ 얼굴 임베딩용 변환기
transformer_Arcface = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [ ]:
# ▶️ SimSwap 옵션 설정
opt = TestOptions()
opt.initialize()
opt.parser.add_argument('-f')  # for Colab compatibility
opt = opt.parse()
opt.isTrain = False
opt.no_simswaplogo = True
opt.use_mask = True
opt.crop_size = crop_size
opt.Arc_path = './arcface_model/arcface_checkpoint.tar'
opt.name = '512' if crop_size == 512 else 'people'
opt.which_epoch = 550000 if crop_size == 512 else 'latest'
mode = 'ffhq' if crop_size == 512 else 'None'

# ▶️ 모델 생성 및 얼굴 인식기 준비
torch.nn.Module.dump_patches = True
model = create_model(opt)
model.eval()

app = Face_detect_crop(name='antelope', root='./insightface_func/models')
app.prepare(ctx_id=0, det_thresh=det_thresh, det_size=det_size, mode=mode)


------------ Options -------------
Arc_path: arcface_model/arcface_checkpoint.tar
aspect_ratio: 1.0
batchSize: 8
checkpoints_dir: ./checkpoints
cluster_path: features_clustered_010.npy
crop_size: 512
data_type: 32
dataroot: ./datasets/cityscapes/
display_winsize: 512
engine: None
export_onnx: None
f: /root/.local/share/jupyter/runtime/kernel-daaa8940-4f44-43e9-936e-9e68d963afd3.json
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
how_many: 50
id_thres: 0.03
image_size: 224
input_nc: 3
instance_feat: False
isTrain: False
label_feat: False
label_nc: 0
latent_size: 512
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
multisepcific_dir: ./demo_file/multispecific
nThreads: 2
n_blocks_global: 6
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 3
n_local_enhancers: 1
name: people
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: False
no_simswaplogo: False
norm: batch
norm_G: spectralspadesyncbatch3x3
ntest: inf
onnx: 

  warnings.warn(



Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: ./insightface_func/models/antelope/glintr100.onnx recognition
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: ./insightface_func/models/antelope/scrfd_10g_bnkps.onnx detection
set det-size: (320, 320)


In [ ]:
import os
import cv2
from PIL import Image
import torch.nn.functional as F

# ▶️ 입력/출력 경로
video_dir = '/content/videos'
photo_dir = '/content/photos'
output_root = '/content/results'

# ▶️ 파일 목록 (정렬된 순서로 매칭)
video_files = sorted([f for f in os.listdir(video_dir) if f.endswith('.mp4')])
photo_files = sorted([f for f in os.listdir(photo_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])

# ▶️ 순서 기준 매칭
num_files = min(len(video_files), len(photo_files))

for i in range(num_files):
    video_file = video_files[i]
    photo_file = photo_files[i]

    video_id = str(i + 1).zfill(4)  # 0001, 0002, ...
    temp_path = os.path.join(output_root, f"tmp_{video_id}")
    output_path = os.path.join(temp_path, f"result_{video_id}.mp4")  # ✅ tmp 폴더 안에 저장

    # ✅ temp 폴더 생성
    os.makedirs(temp_path, exist_ok=True)

    # ✅ 실제 경로 지정
    input_video = os.path.join(video_dir, video_file)
    photo_path = os.path.join(photo_dir, photo_file)

    opt.video_path = input_video
    opt.pic_a_path = photo_path
    opt.output_path = output_path
    opt.temp_path = temp_path

    # ▶ 얼굴 임베딩 생성
    img_a_whole = cv2.imread(photo_path)
    result = app.get(img_a_whole, opt.crop_size)

    if result is None:
        h, w, _ = img_a_whole.shape
        center = (w // 2, h // 2)
        half = opt.crop_size // 2
        cropped = img_a_whole[max(0, center[1]-half):center[1]+half,
                              max(0, center[0]-half):center[0]+half]
        img_pil = Image.fromarray(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))
    else:
        img_align, _ = result
        img_pil = Image.fromarray(cv2.cvtColor(img_align[0], cv2.COLOR_BGR2RGB))

    img_a = transformer_Arcface(img_pil).unsqueeze(0).cuda()
    img_id_downsample = F.interpolate(img_a, size=(112, 112))
    latend_id = model.netArc(img_id_downsample)
    latend_id = F.normalize(latend_id, p=2, dim=1)

    # ▶ 얼굴 교체 실행
    video_swap(input_video, latend_id, model, app, output_path,
               temp_results_dir=temp_path,
               no_simswaplogo=opt.no_simswaplogo,
               use_mask=opt.use_mask,
               crop_size=opt.crop_size)

    print(f"[✅] 완료: {video_id} → {output_path}")


Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


(142, 366, 4)


100%|██████████| 44.7M/44.7M [00:00<00:00, 114MB/s]
  0%|          | 0/469 [00:00<?, ?it/s]WARNING:py.warnings:/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]

100%|██████████| 469/469 [02:17<00:00,  3.41it/s]


Moviepy - Building video /content/results/tmp_0001/result_0001.mp4.
Moviepy - Writing video /content/results/tmp_0001/result_0001.mp4



Moviepy - Done !
Moviepy - video ready /content/results/tmp_0001/result_0001.mp4
[✅] 완료: 0001 → /content/results/tmp_0001/result_0001.mp4
(142, 366, 4)


...

Moviepy - Building video /content/results/tmp_0003/result_0003.mp4.
Moviepy - Writing video /content/results/tmp_0003/result_0003.mp4



Moviepy - Done !
Moviepy - video ready /content/results/tmp_0003/result_0003.mp4
[✅] 완료: 0003 → /content/results/tmp_0003/result_0003.mp4
(142, 366, 4)


100%|██████████| 529/529 [02:35<00:00,  3.41it/s]


Moviepy - Building video /content/results/tmp_0004/result_0004.mp4.
Moviepy - Writing video /content/results/tmp_0004/result_0004.mp4



Moviepy - Done !
Moviepy - video ready /content/results/tmp_0004/result_0004.mp4
[✅] 완료: 0004 → /content/results/tmp_0004/result_0004.mp4


In [ ]:
import os
from glob import glob

# 🔧 경로 설정
output_root = "/content/results"
tmp_dirs = sorted(glob(os.path.join(output_root, "tmp_*")))

print(f"총 {len(tmp_dirs)}개 영상 후처리 시작")

for i, tmp_dir in enumerate(tmp_dirs):
    base_id = tmp_dir.split("_")[-1]  # 'tmp_0001' → '0001'

    result_path  = os.path.join(tmp_dir, f"result_{base_id}.mp4")
    denoise_path = os.path.join(tmp_dir, f"denoise_{base_id}.mp4")
    sharpen_path = os.path.join(tmp_dir, f"sharpen_{base_id}.mp4")
    final_path   = os.path.join(tmp_dir, f"final_{base_id}.mp4")

    print(f"\n▶️ [{i+1}/{len(tmp_dirs)}] result_{base_id}.mp4 후처리 중...")

    if not os.path.exists(result_path):
        print(f"⚠️ result_{base_id}.mp4 없음 → 건너뜀")
        continue

    # 노이즈 제거
    print(" - 노이즈 제거 중...")
    os.system(f'ffmpeg -y -i "{result_path}" -vf "hqdn3d=4.0:3.0:6.0:4.5" -c:a copy "{denoise_path}"')

    # 선명화
    print(" - 선명화 중...")
    os.system(f'ffmpeg -y -i "{denoise_path}" -vf "unsharp=5:5:1.0:5:5:0.0" -c:a copy "{sharpen_path}"')

    # 최종 출력 (30fps + 고화질 압축)
    print(" - 최종 인코딩 중...")
    os.system(f'ffmpeg -y -i "{sharpen_path}" -filter:v "fps=30" -c:v libx264 -preset slow -crf 18 -pix_fmt yuv420p "{final_path}"')

    print(f"✅ 저장 완료 → {final_path}")


총 4개 영상 후처리 시작

▶️ [1/4] result_0001.mp4 후처리 중...
 - 노이즈 제거 중...
 - 선명화 중...
 - 최종 인코딩 중...
✅ 저장 완료 → /content/results/tmp_0001/final_0001.mp4

▶️ [2/4] result_0002.mp4 후처리 중...
 - 노이즈 제거 중...
 - 선명화 중...
 - 최종 인코딩 중...
✅ 저장 완료 → /content/results/tmp_0002/final_0002.mp4

▶️ [3/4] result_0003.mp4 후처리 중...
 - 노이즈 제거 중...
 - 선명화 중...
 - 최종 인코딩 중...
✅ 저장 완료 → /content/results/tmp_0003/final_0003.mp4

▶️ [4/4] result_0004.mp4 후처리 중...
 - 노이즈 제거 중...
 - 선명화 중...
 - 최종 인코딩 중...
✅ 저장 완료 → /content/results/tmp_0004/final_0004.mp4
